### Récupération du dataset

In [1]:
from sklearn import datasets

mnist = datasets.fetch_openml('mnist_784', version=1)
X, y = mnist.data, mnist.target.astype(int)

In [5]:
display(X)
print(y.value_counts(dropna=False))

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


class
1    7877
7    7293
3    7141
2    6990
9    6958
0    6903
6    6876
8    6825
4    6824
5    6313
Name: count, dtype: int64


### Simulation labels manquants

In [9]:
from sklearn.model_selection import train_test_split
import numpy as np

# Séparer un ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Masquer une grande partie des labels (ex: 90% des labels de l'entraînement)
rng = np.random.RandomState(42)
y_train[rng.rand(len(y_train)) < 0.9] = -1  # -1 = étiquette inconnue pour LabelSpreading

print("Labels entraînement :", y_train.value_counts(dropna=False))
print("Labels test",y_test.value_counts(dropna=False))

Labels entraînement : class
-1    50436
 1      653
 3      634
 2      565
 7      561
 9      554
 0      548
 6      525
 4      522
 8      517
 5      485
Name: count, dtype: int64
Labels test class
1    1600
7    1503
3    1433
9    1420
6    1396
2    1380
8    1357
0    1343
4    1295
5    1273
Name: count, dtype: int64


### Entraînement Label Spreading

In [10]:
from sklearn.semi_supervised import LabelSpreading
label_model = LabelSpreading(kernel='knn', n_neighbors=7,max_iter=10)

label_model.fit(X_train,y_train)

LabelSpreading(kernel='knn', max_iter=10)

### Evaluation du modèle

In [11]:
# Prédiction
y_pred = label_model.predict(X_test)

import pandas as pd
display(pd.crosstab(y_test,y_pred,rownames=["Réel"],colnames=["Prédit"]))

from sklearn.metrics import cohen_kappa_score
print ("Coeff de COhen Kappa",cohen_kappa_score(y_test,y_pred))


Prédit,0,1,2,3,4,5,6,7,8,9
Réel,,,,,,,,,,
0,1332,0,3,0,0,1,4,1,1,1
1,0,1591,1,1,1,0,0,4,0,2
2,10,33,1288,3,3,2,8,24,6,3
3,0,6,12,1360,2,14,2,16,9,12
4,3,13,1,0,1209,0,4,6,1,58
5,2,7,0,22,2,1213,17,0,2,8
6,5,2,0,0,2,7,1380,0,0,0
7,1,25,5,0,5,0,0,1445,0,22
8,5,20,6,38,4,27,5,12,1225,15


Coeff de COhen Kappa 0.9516334187548652
